In [310]:
import pandas as pd

df = pd.read_csv("all_results.csv")
# fix up slightly broken csv
df['train_col_sd'] = df['model'].str.extract("col_sd_(\d.0)")
df.train_col_sd = df.train_col_sd.astype(float)

df['additive_importance'] = df['attribution_incorrect.blackbg_withpix_model_accuracy'] - df['attribution_incorrect.blackbg_withattrpix_model_accuracy']

df['subtractive_importance'] = df['attribution_incorrect.truebg_withblackattrpix_model_accuracy'] - df['attribution_incorrect.truebg_withblackpix_model_accuracy']

# Extra columns to help later
nice_names = {"predict_importance_absgrad": "grad. mag.",
              "predict_importance_grad_inp": "grad. $\\times$ input",
              "predict_importance_integrated_gradient": "integrated (b)",
              "predict_importance_integrated_gradient_white": "integrated (w)",
              }
df["Attribution method"] = df["attribution_method"].map(nice_names)

df["\\textbf{p}redicted/\\textbf{t}rue"] = df["use_y_true"].map(lambda x: "t" if x else "p")
df["\\textbf{s}oftmax/\\textbf{l}ogit"] = df["use_softmax"].map(lambda x: "s" if x else "l")

for c in df.columns:
    if "accuracy" in c:
        df[c] = df[c].map(lambda x: x*100.)

df2 = df[(df['dataset'] == "SinglePixelDataset") & (df['train_pos_sd'] == 0) & (df['test_pos_sd'] == 0) & (df['test_col_sd'] == 0) & (df['train_col_sd'] == 0) & (df["data_seed"] == 1) & (df["attribution_method"] != "predict_importance_absgrad_absinp") & 
 (df["model_type"] == "resnet18_3x3") & 
 (df["use_softmax"] == True) &
 (df["use_y_true"] == False) 
]
df2#[['attribution_incorrect.truebg_withblackpix_model_accuracy', 'model']]

,overall.withpix_model_accuracy,overall.attribution_correct_accuracy,overall.nopix_model_accuracy,overall.truebg_withblackpix_model_accuracy,attribution_incorrect.nopix_model_accuracy,attribution_incorrect.withpix_model_accuracy,attribution_incorrect.blackbg_withpix_model_accuracy,attribution_incorrect.blackbg_withattrpix_model_accuracy,attribution_incorrect.truebg_withblackattrpix_model_accuracy,attribution_incorrect.truebg_withblackpix_model_accuracy,...,test_col_sd,attribution_method,use_y_true,use_softmax,model,additive_importance,subtractive_importance,Attribution method,\textbf{p}redicted/\textbf{t}rue,\textbf{s}oftmax/\textbf{l}ogit
800,99.82,99.17,13.76,13.16,12.048193,100.000000,100.000000,8.433735,98.795181,12.048193,...,0.0,predict_importance_absgrad,False,True,/scratch/jsh2/single-pixel-runs/resnet18_3x3-3...,0.915663,0.867470,grad. mag.,p,s
804,99.82,61.87,13.76,13.16,16.338841,99.868870,100.000000,11.591922,99.108314,14.975085,...,0.0,predict_importance_grad_inp,False,True,/scratch/jsh2/single-pixel-runs/resnet18_3x3-3...,0.884081,0.841332,grad. $\times$ input,p,s
812,99.82,99.96,13.76,13.16,25.000000,25.000000,100.000000,75.000000,0.000000,0.000000,...,0.0,predict_importance_integrated_gradient,False,True,/scratch/jsh2/single-pixel-runs/resnet18_3x3-3...,0.250000,0.000000,integrated (b),p,s
816,99.82,97.34,13.76,13.16,92.857143,99.248120,100.000000,100.000000,98.872180,93.609023,...,0.0,predict_importance_integrated_gradient_white,False,True,/scratch/jsh2/single-pixel-runs/resnet18_3x3-3...,0.000000,0.052632,integrated (w),p,s
2900,99.84,98.95,13.04,14.49,19.047619,100.000000,100.000000,12.380952,99.047619,16.190476,...,0.0,predict_importance_absgrad,False,True,/scratch/jsh2/single-pixel-runs/resnet18_3x3-3...,0.876190,0.828571,grad. mag.,p,s
2904,99.84,62.19,13.04,14.49,14.890241,99.709072,100.000000,21.237768,98.492462,21.660936,...,0.0,predict_importance_grad_inp,False,True,/scratch/jsh2/single-pixel-runs/resnet18_3x3-3...,0.787622,0.768315,grad. $\times$ input,p,s
2912,99.84,99.51,13.04,14.49,93.877551,97.959184,100.000000,22.448980,95.918367,83.673469,...,0.0,predict_importance_integrated_gradient,False,True,/scratch/jsh2/single-pixel-runs/resnet18_3x3-3...,0.775510,0.122449,integrated (b),p,s
2916,99.84,99.82,13.04,14.49,77.777778,94.444444,100.000000,83.333333,83.333333,72.222222,...,0.0,predict_importance_integrated_gradient_white,False,True,/scratch/jsh2/single-pixel-runs/resnet18_3x3-3...,0.166667,0.111111,integrated (w),p,s
5000,99.74,99.23,10.49,10.34,10.389610,98.701299,100.000000,7.792208,97.402597,7.792208,...,0.0,predict_importance_absgrad,False,True,/scratch/jsh2/single-pixel-runs/resnet18_3x3-3...,0.922078,0.896104,grad. mag.,p,s
5004,99.74,64.02,10.49,10.34,14.591440,99.444136,96.275709,14.174541,98.999444,14.452474,...,0.0,predict_importance_grad_inp,False,True,/scratch/jsh2/single-pixel-runs/resnet18_3x3-3...,0.821012,0.845470,grad. $\times$ input,p,s


In [297]:
from functools import partial


def format_acc(row, key, inc_std=True):
    mean = row[(key, 'mean')]
    if not inc_std:
        return f"{mean:.1f}%"
    
    std = row[(key, 'std')]
    
    if std != std:
        return f"{mean:.1f}% (tbc)"
    return f"{mean:.1f}% ({std:.1f})"

In [298]:
df2 = df[(df['dataset'] == "SinglePixelDataset") & (df['train_pos_sd'] == 0) & (df['test_pos_sd'] == 0) & (df['test_col_sd'] == 0) & (df['train_col_sd'] == 0) & (df["data_seed"] == 1) & (df["attribution_method"] != "predict_importance_absgrad_absinp") & 
 (df["model_type"] == "resnet18_3x3") & 
 (df["use_softmax"] == True) &
 (df["use_y_true"] == False) 
]

table1 = df2.groupby(["dataset", "data_seed", "train_pos_sd", "train_col_sd", "model_type", "attribution_method", "use_y_true", "use_softmax", "test_col_sd", "test_pos_sd", "Attribution method", "\\textbf{p}redicted/\\textbf{t}rue",
    "\\textbf{s}oftmax/\\textbf{l}ogit"], as_index=False).agg({
    "overall.withpix_model_accuracy": ['mean', 'std'],
    "overall.nopix_model_accuracy": ['mean', 'std'],
    "overall.attribution_correct_accuracy": ['mean', 'std']},)

table1[('Val. acc.', '')] = table1.apply(partial(format_acc, key="overall.withpix_model_accuracy"), axis=1)
table1[('No pixel val. acc.', '')] = table1.apply(partial(format_acc, key="overall.nopix_model_accuracy"), axis=1)
table1[('Pointing acc.', '')] = table1.apply(partial(format_acc, key="overall.attribution_correct_accuracy"), axis=1)

table1 = table1.sort_values(["attribution_method", "model_type"])

latex = table1.to_latex(columns=[
    ("Attribution method", ''),
    ("Val. acc.", ''),
    ("No pixel val. acc.", ''),
    ("Pointing acc.", '')
    ],
    float_format='%.3f',
    caption="Pointing accuracy of different attribution methods. Three models of each type (resnet18 and vgg16) were trained, and results show the mean (and std. dev.) of performance. All models learned to solve the task (near-perfect val. acc.), relying heavily on the discriminative pixel (hence low `No pixel val. acc.'). ",
    label="tab:pointing_acc",
    index=False
    )
print(latex.replace("model_type", "Model").replace("resnet18_3x3", "resnet18").replace("begin{table}", "begin{table}[h]\n\\centering").replace("acc. &","&").replace("acc. \\\\","\\\\").replace("\\textbf{t}rue","").replace("\\textbf{l}ogit","").replace("method &","&").replace("&  &  &  &  &  &  \\\\","&  method & \\textbf{t}rue  & \\textbf{l}ogit & acc. & acc. & acc. \\\\").replace("%", "\\%"))

\begin{table}[h]
\centering
\caption{Pointing accuracy of different attribution methods. Three models of each type (resnet18 and vgg16) were trained, and results show the mean (and std. dev.) of performance. All models learned to solve the task (near-perfect val. acc.), relying heavily on the discriminative pixel (hence low `No pixel val. acc.'). }
\label{tab:pointing_acc}
\begin{tabular}{llll}
\toprule
Attribution & Val. & No pixel val. & Pointing \\
 &  &  &  \\
\midrule
grad. mag. & 99.8\% (0.1) & 12.4\% (1.7) & 99.1\% (0.1) \\
grad. $\times$ input & 99.8\% (0.1) & 12.4\% (1.7) & 62.7\% (1.2) \\
integrated (b) & 99.8\% (0.1) & 12.4\% (1.7) & 98.4\% (2.4) \\
integrated (w) & 99.8\% (0.1) & 12.4\% (1.7) & 98.4\% (1.3) \\
\bottomrule
\end{tabular}
\end{table}



In [299]:
df2 = df[(df['dataset'] == "SinglePixelDataset") & (df['train_pos_sd'] == 0) & (df['test_pos_sd'] == 0) & (df['test_col_sd'] == 0) & (df['train_col_sd'] == 0) & (df["data_seed"] == 1) & (df["attribution_method"] != "predict_importance_absgrad_absinp")]

table1 = df2.groupby(["dataset", "data_seed", "train_pos_sd", "train_col_sd", "model_type", "attribution_method", "use_y_true", "use_softmax", "test_col_sd", "test_pos_sd", "Attribution method", "\\textbf{p}redicted/\\textbf{t}rue",
    "\\textbf{s}oftmax/\\textbf{l}ogit"], as_index=False).agg({
    "overall.withpix_model_accuracy": ['mean', 'std'],
    "overall.nopix_model_accuracy": ['mean', 'std'],
    "overall.attribution_correct_accuracy": ['mean', 'std']},)

table1[('Val. acc.', '')] = table1.apply(partial(format_acc, key="overall.withpix_model_accuracy"), axis=1)
table1[('No pixel val. acc.', '')] = table1.apply(partial(format_acc, key="overall.nopix_model_accuracy"), axis=1)
table1[('Pointing acc.', '')] = table1.apply(partial(format_acc, key="overall.attribution_correct_accuracy"), axis=1)

table1 = table1.sort_values(["attribution_method", "model_type"])

latex = table1.to_latex(columns=[
    ("Attribution method", ''),
    ("model_type", ''),
    ("\\textbf{p}redicted/\\textbf{t}rue", ''),
    ("\\textbf{s}oftmax/\\textbf{l}ogit", ''),
    ("Val. acc.", ''),
    ("No pixel val. acc.", ''),
    ("Pointing acc.", '')
    ],
    float_format='%.3f',
    caption="Pointing accuracy of different attribution methods. Three models of each type (resnet18 and vgg16) were trained, and results show the mean (and std. dev.) of performance. All models learned to solve the task (near-perfect val. acc.), relying heavily on the discriminative pixel (hence low `No pixel val. acc.'). ",
    label="tab:pointing_acc",
    index=False
    )
print(latex.replace("model_type", "Model").replace("resnet18_3x3", "resnet18").replace("begin{table}", "begin{table}[H]\n\\centering").replace("acc. &","&").replace("acc. \\\\","\\\\").replace("\\textbf{t}rue","").replace("\\textbf{l}ogit","").replace("method &","&").replace("&  &  &  &  &  &  \\\\","&  method & \\textbf{t}rue  & \\textbf{l}ogit & acc. & acc. & acc. \\\\").replace("%", "\\%"))

\begin{table}[H]
\centering
\caption{Pointing accuracy of different attribution methods. Three models of each type (resnet18 and vgg16) were trained, and results show the mean (and std. dev.) of performance. All models learned to solve the task (near-perfect val. acc.), relying heavily on the discriminative pixel (hence low `No pixel val. acc.'). }
\label{tab:pointing_acc}
\begin{tabular}{lllllll}
\toprule
Model & Attribution & \textbf{p}redicted/ & \textbf{s}oftmax/ & Val. & No pixel val. & Pointing \\
 &  method & \textbf{t}rue  & \textbf{l}ogit & acc. & acc. & acc. \\
\midrule
resnet18 & grad. mag. & p & l & 99.8\% (0.1) & 12.4\% (1.7) & 97.7\% (0.6) \\
resnet18 & grad. mag. & p & s & 99.8\% (0.1) & 12.4\% (1.7) & 99.1\% (0.1) \\
resnet18 & grad. mag. & t & l & 99.8\% (0.1) & 12.4\% (1.7) & 97.7\% (0.6) \\
resnet18 & grad. mag. & t & s & 99.8\% (0.1) & 12.4\% (1.7) & 99.1\% (0.1) \\
vgg16 & grad. mag. & p & l & 99.9\% (0.0) & 11.5\% (1.9) & 100.0\% (0.0) \\
vgg16 & grad. mag. & p & 

In [300]:
df3 = df2[(df2['Attribution method'].str.contains("integrated")) & (df2['use_y_true'] == False) & (df2['use_softmax'] == True)]

table2App = df3.groupby(["dataset", "data_seed", "train_pos_sd", "train_col_sd", "model_type", "attribution_method", "Attribution method", "use_y_true", "use_softmax", "test_col_sd", "test_pos_sd", "\\textbf{p}redicted/\\textbf{t}rue",
    "\\textbf{s}oftmax/\\textbf{l}ogit"], as_index=False).agg({
    "overall.attribution_incorrect": ['mean'],
    "attribution_incorrect.blackbg_withpix_model_accuracy": ["mean", "std"],
    "attribution_incorrect.blackbg_withattrpix_model_accuracy": ["mean", "std"],
    "attribution_incorrect.truebg_withblackpix_model_accuracy": ["mean", "std"],
    "attribution_incorrect.truebg_withblackattrpix_model_accuracy": ["mean", "std"]})

table2App[('additive/discriminative', '')] = table2App.apply(partial(format_acc, key="attribution_incorrect.blackbg_withpix_model_accuracy", inc_std=False), axis=1)
table2App[('additive/attributed', '')] = table2App.apply(partial(format_acc, key="attribution_incorrect.blackbg_withattrpix_model_accuracy", inc_std=False), axis=1)
table2App[('subtractive/discriminative', '')] = table2App.apply(partial(format_acc, key="attribution_incorrect.truebg_withblackpix_model_accuracy", inc_std=False), axis=1)
table2App[('subtractive/attributed', '')] = table2App.apply(partial(format_acc, key="attribution_incorrect.truebg_withblackattrpix_model_accuracy", inc_std=False), axis=1)

table2App = table2App.sort_values(["attribution_method", "model_type"])

latex = table2App.to_latex(columns=[
    ("Attribution method", ''),
    ("model_type", ''),
    ("additive/discriminative", ''),
    ("additive/attributed", ''),
    ("subtractive/discriminative", ''),
    ("subtractive/attributed", '')
    ],
    float_format='%.3f',
    caption="Analysis of integrated gradients when it fails to point at the discriminative pixel. These results explore how important the discriminative pixel is compared to the attributed pixel. Because of the way the models learn, we cannot guarantee that the discriminative pixel is most important, however any reasonable attribution method should at least find a pixel that is at least as important as the discriminative pixel. These results show this is not the case as the attributed accuracies are worse than the discriminative ones (e.g. lower for additive, higher for subtractive).", label="tab:importance-full",
    index=False
    )
print(latex.replace("model_type", "Model").replace("resnet18_3x3", "resnet18").replace("\\toprule","\\toprule\n&& \multicolumn{2}{c}{Additive} & \multicolumn{2}{c}{Subtractive} \\\\\n").replace("additive/", "").replace("subtractive/","").replace("begin{table}", "begin{table}\n\\centering").replace(" &  &  &  &  &  \\\\","").replace("%", "\\%"))

\begin{table}
\centering
\caption{Analysis of integrated gradients when it fails to point at the discriminative pixel. These results explore how important the discriminative pixel is compared to the attributed pixel. Because of the way the models learn, we cannot guarantee that the discriminative pixel is most important, however any reasonable attribution method should at least find a pixel that is at least as important as the discriminative pixel. These results show this is not the case as the attributed accuracies are worse than the discriminative ones (e.g. lower for additive, higher for subtractive).}
\label{tab:importance-full}
\begin{tabular}{llllll}
\toprule
&& \multicolumn{2}{c}{Additive} & \multicolumn{2}{c}{Subtractive} \\

Model & Attribution method & discriminative & attributed & discriminative & attributed \\

\midrule
resnet18 & integrated (b) & 100.0\% & 62.2\% & 60.6\% & 64.8\% \\
vgg16 & integrated (b) & 100.0\% & 94.1\% & 95.2\% & 99.3\% \\
resnet18 & integrated (w) &

In [312]:
df3 = df[(df['dataset'] == "SinglePixelDataset") & (df['train_pos_sd'] == 0) & (df['test_pos_sd'] == 0) & (df['test_col_sd'] == 0) & (df['train_col_sd'] == 0) & (df["data_seed"] == 1) & (df["attribution_method"] != "predict_importance_absgrad_absinp")]


table2App = df3.groupby(["dataset", "data_seed", "train_pos_sd", "train_col_sd", "model_type", "attribution_method", "Attribution method", "use_y_true", "use_softmax", "test_col_sd", "test_pos_sd", "\\textbf{p}redicted/\\textbf{t}rue",
    "\\textbf{s}oftmax/\\textbf{l}ogit"], as_index=False).agg({
    "overall.attribution_incorrect": ['mean'],
    "attribution_incorrect.blackbg_withpix_model_accuracy": ["mean", "std"],
    "attribution_incorrect.blackbg_withattrpix_model_accuracy": ["mean", "std"],
    "attribution_incorrect.truebg_withblackpix_model_accuracy": ["mean", "std"],
    "attribution_incorrect.truebg_withblackattrpix_model_accuracy": ["mean", "std"]})

table2App[('additive/discriminative', '')] = table2App.apply(partial(format_acc, key="attribution_incorrect.blackbg_withpix_model_accuracy", inc_std=False), axis=1)
table2App[('additive/attributed', '')] = table2App.apply(partial(format_acc, key="attribution_incorrect.blackbg_withattrpix_model_accuracy", inc_std=False), axis=1)
table2App[('subtractive/discriminative', '')] = table2App.apply(partial(format_acc, key="attribution_incorrect.truebg_withblackpix_model_accuracy", inc_std=False), axis=1)
table2App[('subtractive/attributed', '')] = table2App.apply(partial(format_acc, key="attribution_incorrect.truebg_withblackattrpix_model_accuracy", inc_std=False), axis=1)

table2App = table2App.sort_values(["attribution_method", "model_type"])

latex = table2App.to_latex(columns=[
    ("Attribution method", ''),
    ("model_type", ''),
    ("\\textbf{p}redicted/\\textbf{t}rue", ''),
    ("\\textbf{s}oftmax/\\textbf{l}ogit", ''),
    ("additive/discriminative", ''),
    ("additive/attributed", ''),
    ("subtractive/discriminative", ''),
    ("subtractive/attributed", '')
    ],
    float_format='%.3f',
    caption="Analysis of integrated gradients when it fails to point at the discriminative pixel. These results explore how important the discriminative pixel is compared to the attributed pixel. Because of the way the models learn, we cannot guarantee that the discriminative pixel is most important, however any reasonable attribution method should at least find a pixel that is at least as important as the discriminative pixel. These results show this is not the case.", label="tab:importance-full",
    index=False
    )
print(latex.replace("model_type", "Model").replace("resnet18_3x3", "resnet18").replace("\\toprule","\\toprule\n&&\\textbf{p}redicted/&\\textbf{s}oftmax/& \multicolumn{2}{c}{Additive} & \multicolumn{2}{c}{Subtractive} \\\\\n").replace("additive/", "").replace("subtractive/","").replace("begin{table}", "begin{table}[H]\n\\centering").replace("&  &  &  &  &  &  &  \\\\","").replace("& discriminative","& discr.").replace("& attributed","& attrd.").replace("& \\textbf{p}redicted/","& ").replace("& \\textbf{s}oftmax/","& ").replace("%","\\%"))

\begin{table}[H]
\centering
\caption{Analysis of integrated gradients when it fails to point at the discriminative pixel. These results explore how important the discriminative pixel is compared to the attributed pixel. Because of the way the models learn, we cannot guarantee that the discriminative pixel is most important, however any reasonable attribution method should at least find a pixel that is at least as important as the discriminative pixel. These results show this is not the case.}
\label{tab:importance-full}
\begin{tabular}{llllllll}
\toprule
&&\textbf{p}redicted/&\textbf{s}oftmax/& \multicolumn{2}{c}{Additive} & \multicolumn{2}{c}{Subtractive} \\

Model & Attribution method & \textbf{t}rue & \textbf{l}ogit & discr. & attrd. & discr. & attrd. \\
 
\midrule
resnet18 & grad. mag. & p & l & 100.0\% & 12.1\% & 17.4\% & 99.2\% \\
resnet18 & grad. mag. & p & s & 100.0\% & 9.5\% & 12.0\% & 98.4\% \\
resnet18 & grad. mag. & t & l & 100.0\% & 12.1\% & 17.4\% & 98.9\% \\
resnet18 & g